# User Access Token

## User Acces Token

In [0]:
# from azure.keyvault import KeyVaultClient
import requests
# import pyotp

In [0]:
## Authenticate to Keyvaults
key_Vault_name = "ia-aat"


In [0]:
grant_type = "password"
client_id = dbutils.secrets.get(key_Vault_name, "idam-client-id")
client_secret = dbutils.secrets.get(key_Vault_name, "idam-secret")
username = dbutils.secrets.get(key_Vault_name, "system-username")
password = dbutils.secrets.get(key_Vault_name, "system-password")
scope = "openid profile roles"

In [0]:
# base_url = "https://idam-api.aat.platform.hmcts.net/o/token"
idam_host = "https://idam-web-public.aat.platform.hmcts.net"
base_url = f"{idam_host}/o/token"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

In [0]:
data = {
    "grant_type": "password",
    "client_id": client_id,
    "client_secret": client_secret,
    "username": username,
    "password": password,
    "scope": "openid profile roles"
}

# data = {
#     "grant_type": "password",
#     "client_id": "iac",
#     "client_secret": "thUphEveC2Ekuqedaneh4jEcRuba4t2t",
#     "username": "iac.idamdashboard.aat@mailinator.com",
#     "password": "AldgateT0wer",
#     # "scope": "openid profile roles"
# }
data

In [0]:
def generate_user_token(env):
    #----# determine the instance and Keyvault to use #-----#
    if env == "sbox":
        idam_host = "https://idam-web-public.aat.platform.hmcts.net"
        key_Vault_name = "ia-aat"
    if env == "stg":
        idam_host = "https://idam-web-public.aat.platform.hmcts.net"
        key_Vault_name = "ia-aat"
    if env == "prod":
        idam_host = " "
        key_Vault_name = ""

    base_url = f"{idam_host}/o/token"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    #-----# Get variables from keyvault #-----#
    grant_type = "password"
    client_id = dbutils.secrets.get(key_Vault_name, "idam-client-id")
    client_secret = dbutils.secrets.get(key_Vault_name, "idam-secret")
    username = dbutils.secrets.get(key_Vault_name, "system-username")
    password = dbutils.secrets.get(key_Vault_name, "system-password")
    scope = "openid profile roles"

    #-----# Create body of the message #-----#
    data = {
        "grant_type": "password",
        "client_id": client_id,
        "client_secret": client_secret,
        "username": username,
        "password": password,
        "scope": "openid profile roles"
    }
    #-----# POST Request to get user access token #-----#
    try:
        idam_response = requests.post(base_url, headers=headers, data=data)
        # Check and handle response
        if idam_response.status_code == 200:
            access_token = idam_response.json()["access_token"]
            print("Succesfully acquired: User Access Token")
            return access_token
        else:
            print("Failed to get token:", idam_response.status_code)
            print(idam_response.text)
    except Exception as e:
        print("Error:", e)
        return None

generate_user_token("sbox")



## Service to Service Token

In [0]:
s2s_base_url = "http://rpe-service-auth-provider-aat.service.core-compute-aat.internal"

In [0]:

s2s_url = "http://rpe-service-auth-provider-aat.service.core-compute-aat.internal"
s2s_secret = "AAAAAAAAAAAAAAAC"
s2s_microservice = "iac"

In [0]:
otp = pyotp.TOTP(s2s_secret).now()

otp

In [0]:
s2s_payload = {
    "microservice": s2s_microservice,
    "oneTimePassword": otp
}

In [0]:
s2s_response = requests.post(
    s2s_base_url,json=s2s_payload)

# CCD API

## Start Case Creation

In [0]:
uid = "get from idam response, user id/user info"
jid = "IA"
ctid = "Asylum"
etid = "ariaCreateCase"



In [0]:
ccd_base_url = "ccd-data-store-api-ia-case-api-pr-2660.preview.platform.hmcts.net"
start_case_creation = f"/caseworkers/{uid}/jurisdictions/{jid}/case-types/{ctid}/event-triggers/{etid}/token"

In [0]:
ccd_start_case_creation_response = requests.get(
    ccd_base_url+start_case_creation, 
    headers={

    "Authorization": f"Bearer {idam_response.json()['access_token']}",

    "ServiceAuthorization": f"Bearer {s2s_response.json()['access_token']}"
}
    )


In [0]:
## Get the event id and event token from the response
event_token = ccd_start_case_creation_response.json()["token"]
event_id = ccd_start_case_creation_response.json()["eventId"]

## Validate Case

In [0]:
case_payload_example = ""

In [0]:
case_data = {
    "event_token": event_token,
    "event": {
        "id": "startAppeal",
        "summary": "Starting appeal",
        "description": "Validate draft appeal"
    },
    "data": case_payload_example
}


In [0]:
ccd_validate_case_ep = f"/caseworkers/{uid}/jurisdictions/{jid}/case-types/{ctid}/validate"

In [0]:
ccd_validate_case_response = requests.post(ccd_base_url+ccd_validate_case_ep, json=case_data, headers={
    "Authorization": f"Bearer {idam_response.json()['access_token']}")

## Submit Case Creation